In [2]:
from google.colab import drive
drive.mount('/drive')
!pip install faker
!pip install bcrypt
!pip install bson

Drive already mounted at /drive; to attempt to forcibly remount, call drive.mount("/drive", force_remount=True).
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for bson: filename=bson-0.5.10-py3-none-any.whl size=11975 sha256=a4f67f43b017b3ef60b687b8a8d0a3e6f755d705172e6a0bf0574fa88828b7ce
  Stored in directory: /root/.cache/pip/wheels/cb/f3/45/c859e83339943dfe2f43e1c9aaebdc00db321191a6fe120947
Successfully built bson


In [3]:
import os
import pandas as pd
import random
import json
from faker import Faker
from bcrypt import hashpw, gensalt
from bson import ObjectId

# Initialize Faker for generating fake names, emails, etc.
fake = Faker()

# Function to hash passwords using bcrypt (with a low number of rounds for speed)
def hash_password(password: str) -> str:
    return hashpw(password.encode('utf-8'), gensalt(rounds=4)).decode('utf-8')

def generate_buyers_from_properties(properties_file: str, output_file: str, total_buyers: int = 10000):
    """
    Generates buyers based on the properties listed in the given CSV file.
    Each buyer will have favorite properties chosen from a single city.

    Args:
        properties_file (str): Path to the properties_on_sale CSV file.
        output_file (str): Path to save the generated buyers CSV file.
        total_buyers (int): Total number of buyers to generate (default is 10,000).
    """
    # Load the properties CSV into a DataFrame
    properties_df = pd.read_csv(properties_file)

    # Group properties by city, retaining a summary of each property
    city_properties = {}
    for _, row in properties_df.iterrows():
        city = row['city']
        # Create a summary of the property with the essential fields
        summary = {
            '_id': row['_id'],           # Keep the original property ID
            'thumbnail': row['thumbnail'],
            'address': row['address'],
            'price': row['price'],
            'area': row['area']
        }
        # Append the summary to the list for that city
        if city not in city_properties:
            city_properties[city] = []
        city_properties[city].append(summary)

    # List to hold all generated buyers
    buyers = []
    # List of all available cities (keys from the grouped properties)
    available_cities = list(city_properties.keys())

    # Generate the specified number of buyers
    for i in range(total_buyers):
        # Randomly select a city so that all favorites come from that city
        chosen_city = random.choice(available_cities)
        properties_in_city = city_properties[chosen_city]

        # Generate fake first and last names for the buyer
        fake_first_name = fake.first_name()
        fake_last_name = fake.last_name()

        # Create a buyer record with a unique ObjectId and fake data
        buyer = {
            '_id': str(ObjectId()),
            'name': fake_first_name,
            'surname': fake_last_name,
            'email': fake_first_name.lower() + fake_last_name.lower() + '@buyer.com',
            'password': hash_password(fake.password()),
            'phone_number': fake.phone_number(),
            'favourites': []  # This will hold favorite properties from the chosen city
        }

        # If properties exist in the chosen city, randomly select between 1 and 3 as favorites
        if properties_in_city:
            num_favs = random.randint(1, min(3, len(properties_in_city)))
            buyer['favourites'] = random.sample(properties_in_city, num_favs)

        buyers.append(buyer)

    # Convert the list of buyers to a DataFrame
    buyers_df = pd.DataFrame(buyers)
    # Convert the list of favorite properties to a JSON string for CSV export
    buyers_df['favourites'] = buyers_df['favourites'].apply(json.dumps)
    buyers_df.to_csv(output_file, index=False)
    print(f"Generated {total_buyers} buyers and saved to {output_file}")

if __name__ == "__main__":
    # Define the input file (final properties on sale CSV) and the output file path for buyers
    properties_file_path = '/drive/MyDrive/Intermediate Corporation/Dati/Zillow/Formatted/properties_on_sale.csv'
    buyers_output_file_path = '/drive/MyDrive/Intermediate Corporation/Dati/Zillow/Formatted/buyers_2.csv'

    # Generate 10,000 buyers from the properties file
    generate_buyers_from_properties(properties_file_path, buyers_output_file_path, total_buyers=10000)


Generated 10000 buyers and saved to /drive/MyDrive/Intermediate Corporation/Dati/Zillow/Formatted/buyers_2.csv
